In [1]:
import torch
import torch.nn as nn
import torch.utils.data as utildata
import torchvision.datasets as ds
import torchvision.transforms as transforms
from google.colab import drive

drive.mount('/content/gdrive')

import os

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# 디바이스 선택과 데이터셋 init
---
torch.device -> 사용할 device 선택하도록

ds.MNIST -> MNIST dataset을 Tensor 타입으로 받으며 없으면 다운로드

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # device 선택 (현재 CUDA 사용 가능하면 쓰고, 아니면 cpu로)
# init MNIST dataset
trainSet = ds.MNIST(root='./data/',
                    train=True,
                    transform=transforms.ToTensor(),
                    download=True)

testSet = ds.MNIST(root='./data/',
                   train=False,
                   transform=transforms.ToTensor(),
                   download=True)

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ./data/
    Transforms (if any): ToTensor()
    Target Transforms (if any): None


#Hypoer parameter
---
일종의 튜닝 옵션

train_epochs -> 에포크는 학습 반복 횟수

learning_rate -> 기울기 찾을때 이동하는 속도

batch_size -> 한번에 불러올 데이터(weight 갱신 주기?)

classes -> MNIST는 0~9이므로 10개다.


In [0]:
# hyper param
train_epochs = 10 # train 몇번 할거냐
learning_rate = 0.001 # 옵티마이저에서 iteration 마다 이동하는 속도?? 뭐라고 해야하지 쨌든 너무 크면 위로 날아가고 너무 작으면 최소 로스 찾기전에 끝남
batch_size = 20 # 전체 데이터에서 일정 수의 샘플로 나눠서 weight 갱신 (여기선 20번마다)
classes = 10 # 0~9

# 데이터셋 불러오는 부분
---
dataset -> 불러올 데이터셋

batch_size -> 위 설명대로

shuffle -> training할때 마다 데이터가 새로 배열된다. 특정 data에만 종속되어 학습되게하는 overfitting을 피할 수 있다고 함

num_workers -> 서브 프로세스 얼마나 돌릴건지

In [8]:
# data loader
train = utildata.DataLoader(dataset=trainSet, # 불러올 데이타셋
                           batch_size=batch_size,
                           shuffle=True, # 매 epoch마다 데이터가 재배열 -> 매 epoch마다 dataset이 섞이기 때문에 overfitting을 피할 수 있음
                           num_workers=4) # 데이터 불러올때 서브 프로세스 몇개 돌릴거냐? 0은 메인 프로세스에서만
test = utildata.DataLoader(dataset=testSet,
                           batch_size=batch_size,
                           shuffle=False,
                           num_workers=4)


# CNN 모델 생성
---
기본적으로 torch.nn 내에 있는 모듈을 상속받아 서브클래스를 만들어 사용한다.

__init__에서 사용할 멤버변수를 forward에서는 정방향으로 연산한다.

Sequential 컨테이너는 들어갈 모듈의 순서를 나타낸다.

CNN은 기본적으로 (입력 채널 수, 출력 채널 수, 커널 사이즈(5x5), stride(커널 몇칸씩 이동할 건지), padding(좌우상하 2칸씩)) 을 받는다.

BatchNorm2d는 activation function에 입력값을 넣기전에 평균이 0으로 이쁜 모양으로 만들어 주기 위한 기법 (기울기 실종 등 잡아줌)

ReLU는 Activation function

MaxPool2d는 이제 Convolution layer가 끝난 뒤 풀링하는 단계다 maxpooling이면 가장 큰 값 풀링

fc는 fully connected layer로 cnn을 통해 feature값을 생성하면 합쳐서 classification

In [0]:
# Convolutional NN Midel
class myCNN(nn.Module):
  def __init__(self, classes = 10):
    super(myCNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer2 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
    self.fc = nn.Linear(7*7*32, classes)
  def forward(self, input):
    res = self.layer1(input)
    res = self.layer2(res)
    res = res.reshape(res.size(0), -1)
    res = self.fc(res)
    return res

# Optimizer
---
이제 어떻게 최적의 방향으로 갈건지 정하는 방향이다.

실제 label과 predict값이 얼마나 차이가 나느냐를 loss라고 하는데 이 loss를 최소화 하기 위해 어떠한 방향으로? 갈건지 정하는 알고리즘?

In [10]:
model = myCNN(classes).to(device)
print(model)
# optimizer (Gradient Descent, Stochastic Gradientt Descent,
# Adaptive Gradient, Adaptive Delta, Adaptive Moment Estimation ... )
sgd = torch.optim.SGD(model.parameters(), lr=learning_rate)
adagrad = torch.optim.Adagrad(model.parameters(), lr=learning_rate)
adadelta = torch.optim.Adadelta(model.parameters(), lr=learning_rate)
adam = torch.optim.Adam(model.parameters(), lr=learning_rate)

myCNN(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)


# loss function
---
위에서 loss에 대해 간단하게 말했는데, 이 loss를 구하는 식

crossEntropy가 어떻게 하는거지 까먹음

In [0]:
# loss function (negative log-likelihood)
crossEn = nn.CrossEntropyLoss()

# Train
---
실제로 학습하는 부분이다. 지금까지 위에서 정의한거 다 쓰는곳임
(CNN의 경우 228초 경과)

In [0]:
import time

st = time.time()
for epoch in range(train_epochs):
    for i, (images,labels) in enumerate(train):
      images = images.to(device)
      labels = labels.to(device)
      predic = model(images)
      loss = crossEn(predic,labels)
      adam.zero_grad()
      loss.backward()
      adam.step()

      if(i+1)%1000 == 0:
        print('Epoch: [{}/{}], Loss: {:.4f}, Step: {}'.format(epoch+1, train_epochs,loss.item(),i+1))
print("--- %s seconds ---" %(time.time() - st))

Epoch: [1/10], Loss: 0.0000, Step: 1000
Epoch: [1/10], Loss: 0.0104, Step: 2000
Epoch: [1/10], Loss: 0.0001, Step: 3000
Epoch: [2/10], Loss: 0.0001, Step: 1000
Epoch: [2/10], Loss: 0.0000, Step: 2000
Epoch: [2/10], Loss: 0.1420, Step: 3000
Epoch: [3/10], Loss: 0.0000, Step: 1000
Epoch: [3/10], Loss: 0.0000, Step: 2000
Epoch: [3/10], Loss: 0.0017, Step: 3000
Epoch: [4/10], Loss: 0.0003, Step: 1000
Epoch: [4/10], Loss: 0.0000, Step: 2000
Epoch: [4/10], Loss: 0.0001, Step: 3000
Epoch: [5/10], Loss: 0.0001, Step: 1000
Epoch: [5/10], Loss: 0.0078, Step: 2000
Epoch: [5/10], Loss: 0.0001, Step: 3000
Epoch: [6/10], Loss: 0.0000, Step: 1000
Epoch: [6/10], Loss: 0.0085, Step: 2000
Epoch: [6/10], Loss: 0.0001, Step: 3000
Epoch: [7/10], Loss: 0.0001, Step: 1000
Epoch: [7/10], Loss: 0.0000, Step: 2000
Epoch: [7/10], Loss: 0.0001, Step: 3000
Epoch: [8/10], Loss: 0.0000, Step: 1000
Epoch: [8/10], Loss: 0.0000, Step: 2000
Epoch: [8/10], Loss: 0.0000, Step: 3000
Epoch: [9/10], Loss: 0.0000, Step: 1000


# Train(2)
---
이건 Colab용으로..

In [0]:
if os.path.isfile('./gdrive/My Drive/Colab Notebooks/model/myCNN.pkl'):
  # cpu ver
  #model.load_state_dict(torch.load('myNN.pkl',map_location='cpu'))
  # gpu ver
  model.load_state_dict(torch.load('./gdrive/My Drive/Colab Notebooks/model/myCNN.pkl'))
else:
  for epoch in range(train_epochs):
    for i, (images,labels) in enumerate(train):
      images = images.to(device)
      labels = labels.to(device)
      predic = model(images)
      loss = crossEn(predic,labels)
      adam.zero_grad()
      loss.backward()
      adam.step()

      if(i+1)%100 == 0:
        print('Epoch: [{}/{}], Loss: {:.4f}, Step: {}'.format(epoch+1, train_epochs,loss.item(),i+1))
  torch.save(model.state_dict(), './gdrive/My Drive/Colab Notebooks/model/myCNN.pkl')

Epoch: [1/10], Loss: 0.1200, Step: 100
Epoch: [1/10], Loss: 0.2459, Step: 200
Epoch: [1/10], Loss: 0.2656, Step: 300
Epoch: [1/10], Loss: 0.0315, Step: 400
Epoch: [1/10], Loss: 0.0428, Step: 500
Epoch: [1/10], Loss: 0.0476, Step: 600
Epoch: [1/10], Loss: 0.0822, Step: 700
Epoch: [1/10], Loss: 0.0438, Step: 800
Epoch: [1/10], Loss: 0.1536, Step: 900
Epoch: [1/10], Loss: 0.0363, Step: 1000
Epoch: [1/10], Loss: 0.0075, Step: 1100
Epoch: [1/10], Loss: 0.1943, Step: 1200
Epoch: [1/10], Loss: 0.1242, Step: 1300
Epoch: [1/10], Loss: 0.0439, Step: 1400
Epoch: [1/10], Loss: 0.0021, Step: 1500
Epoch: [1/10], Loss: 0.0085, Step: 1600
Epoch: [1/10], Loss: 0.0502, Step: 1700
Epoch: [1/10], Loss: 0.0010, Step: 1800
Epoch: [1/10], Loss: 0.0103, Step: 1900
Epoch: [1/10], Loss: 0.1109, Step: 2000
Epoch: [1/10], Loss: 0.0059, Step: 2100
Epoch: [1/10], Loss: 0.1442, Step: 2200
Epoch: [1/10], Loss: 0.0077, Step: 2300
Epoch: [1/10], Loss: 0.0054, Step: 2400
Epoch: [1/10], Loss: 0.0009, Step: 2500
Epoch: [1

# Test
---
이제 얼마나 잘 학습했는지 test를 하면 된다. (Accr : 99.22)

In [0]:
model.eval()
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test:
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  print('Accuracy test on the 10000 images : {}'.format(100 * correct/total))

Accuracy test on the 10000 images : 99.22
